In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

2023-05-29 17:26:07.059721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# 전처리할 텍스트
text = "여의도공원, 샛강공원, 여의도한강시민공원은 모두 여의도에 있다."

In [6]:
tokens = text_to_word_sequence(text)
text, tokens

('여의도공원, 샛강공원, 여의도한강시민공원은 모두 여의도에 있다.',
 ['여의도공원', '샛강공원', '여의도한강시민공원은', '모두', '여의도에', '있다'])

### bag-of-words

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
docs = [
    '여의도역은 여의도에 있다',
    '여의도에 여의도역이 있다',
    '서울에 여의도역이 있다',
    '어쩌면 서울에 여의도가 있을 수 있다'
]

In [16]:
# 단어 수
token = Tokenizer()
token.fit_on_texts(docs)
token.word_counts

OrderedDict([('여의도역은', 1),
             ('여의도에', 2),
             ('있다', 4),
             ('여의도역이', 2),
             ('서울에', 2),
             ('어쩌면', 1),
             ('여의도가', 1),
             ('있을', 1),
             ('수', 1)])

In [18]:
# 문장 수
token.document_count

4

In [19]:
# 각 단어 별 들어 있는 문장 수
token.word_docs

defaultdict(int,
            {'여의도역은': 1,
             '있다': 4,
             '여의도에': 2,
             '여의도역이': 2,
             '서울에': 2,
             '있을': 1,
             '수': 1,
             '어쩌면': 1,
             '여의도가': 1})

## Token one-hot

In [30]:
# 텍스트 토큰 to int value by one-hot

# 대상 텍스트
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'

In [31]:
# tokenizing
token = Tokenizer()
token.fit_on_texts([text])

In [32]:
# word index
token.word_index

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}

In [33]:
# word index로 바꿔주는 함수 사용
x = token.texts_to_sequences([text])
x

[[1, 2, 3, 4, 5, 6]]

In [34]:
from tensorflow.keras.utils import to_categorical

# one-hot 결과 만들기
# 주의 : index는 0부터 시작하므로, word_size는 +1 해줘야함
word_size = len(token.word_index) + 1
x = to_categorical(x, num_classes=word_size)

x

array([[[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]], dtype=float32)

### 단어 임베딩word embedding

In [37]:
from tensorflow.keras.layers import Embedding
# Embedding(16, 4)
"""
입력 16개 단어
출력 4개 벡터
"""

'\n입력 16개 단어\n출력 4개 벡터\n'

## 자연어 처리 실습

In [52]:
import numpy as np

# 영화 리뷰(리뷰, 긍정/부정 여부)
reviews = np.array([
    ('너무 재미있어요', 1),
    ('최고에요', 1),
    ('참 잘 만든 영화에요', 1),
    ('추천하고 싶은 영화입니다', 1),
    ('한 번 더 보고싶네요', 1),
    ('별로에요', 0),
    ('글쎄요', 0),
    ('생각보다 지루하네요', 0),
    ('연기가 어색해요', 0),
    ('재미없어요', 0)
])

In [93]:
# 리뷰와 긍정/부정 데이터 분리
docs = reviews[:, 0]
clazz = reviews[:, 1].astype(int)

docs, clazz

(array(['너무 재미있어요', '최고에요', '참 잘 만든 영화에요', '추천하고 싶은 영화입니다', '한 번 더 보고싶네요',
        '별로에요', '글쎄요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요'], dtype='<U21'),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0]))

In [55]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
x = token.texts_to_sequences(docs)
x

[[1, 2],
 [3],
 [4, 5, 6, 7],
 [8, 9, 10],
 [11, 12, 13, 14],
 [15],
 [16],
 [17, 18],
 [19, 20],
 [21]]

In [74]:
# 패딩padding
"""
토큰화 햇을 때, 각 데이터 길이가 다른 것을 같게 만들어주는 과정
"""
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_x = pad_sequences(x, 4)
padded_x

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [11, 12, 13, 14],
       [ 0,  0,  0, 15],
       [ 0,  0,  0, 16],
       [ 0,  0, 17, 18],
       [ 0,  0, 19, 20],
       [ 0,  0,  0, 21]], dtype=int32)

In [86]:
# 임베딩 시, 입력, 출력, 매 시행 입력 수 정해야함
# 입력 : 전체 단어 수
word_size = len(token.word_index) + 1
# 출력 : hyperparamter, hidden layer의 노드와 유사한 역할을 한다
output_size = 8
# 매 시행 입력 수 : padded word size
iter_size = len(padded_x[0])

word_size, output_size, iter_size

(22, 8, 4)

## 자연어 딥러닝 모델 학습

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

import numpy as np

In [88]:
# Embedding 후 Flatten 해서 1차원 만들고 이를 돌려서 sigmoid로 0 / 1 구분
model = Sequential([
    Embedding(word_size, output_size, input_length=iter_size),
    Flatten(),
    Dense(1, activation='sigmoid')
])
model.summary()

# 긍 / 부 구분이므로 binary_crossentropy
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              176       
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [96]:
model.fit(padded_x, clazz, epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1880 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 0.1868 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1855 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1842 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1830 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1817 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1805 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1793 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 6ms/step - loss: 0.1781 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 0.1769 - accuracy: 1.0000
Epoch 11/50
1/1 [======================

In [125]:
# 학습 결과 확인
for i in range(len(padded_x)):
    [[result]] = model.predict(np.array([padded_x[i]]), verbose=0)
    result = 1 if result > 0.5 else 0
    
    print(f'%ith data predict : %i, real: %i', i, result, clazz[i])

%ith data predict : %i, real: %i 0 1 1
%ith data predict : %i, real: %i 1 1 1
%ith data predict : %i, real: %i 2 1 1
%ith data predict : %i, real: %i 3 1 1
%ith data predict : %i, real: %i 4 1 1
%ith data predict : %i, real: %i 5 0 0
%ith data predict : %i, real: %i 6 0 0
%ith data predict : %i, real: %i 7 0 0
%ith data predict : %i, real: %i 8 0 0
%ith data predict : %i, real: %i 9 0 0
